In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Google Drive'ı bağla
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive başarıyla bağlandı! ✅")

Mounted at /content/drive
Google Drive başarıyla bağlandı! ✅


In [ ]:

# 📌 Veri seti yolları
train_dir = '/content/drive/MyDrive/veriseti/prcc/rgb/train'
val_dir = '/content/drive/MyDrive/veriseti/prcc/rgb/val'

# 📌 Veri yükleme ve ön işleme
batch_size = 32
img_size = (224, 224)

print("\nVeri seti yükleniyor... 📂")
train_dataset = image_dataset_from_directory(
    train_dir, image_size=img_size, batch_size=batch_size, label_mode='int')

val_dataset = image_dataset_from_directory(
    val_dir, image_size=img_size, batch_size=batch_size, label_mode='int')

# 📌 Sınıf isimlerini al ve sınıf sayısını belirle
class_names = train_dataset.class_names  # Sınıf isimlerini al
num_classes = len(class_names)  # 📌 Sınıf sayısını al

print(f"Toplam {num_classes} sınıf bulundu: {class_names[:5]}... ✅")



Veri seti yükleniyor... 📂
Found 17896 files belonging to 150 classes.
Found 5002 files belonging to 150 classes.
Toplam 150 sınıf bulundu: ['092', '095', '098', '102', '104']... ✅


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ** Veri Artırma İşlemi (Data Augmentation)**
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),  # Yatay çevirme
    tf.keras.layers.RandomRotation(0.2),  # Döndürme oranını artır (0.2 = 72 derece)
    tf.keras.layers.RandomZoom(0.2),  # Zoom oranını artır
    tf.keras.layers.RandomBrightness(0.5),  # Parlaklık değişimini artır
    tf.keras.layers.RandomContrast(0.6),  # Kontrast değişimini artır
    tf.keras.layers.GaussianNoise(0.1)  # Hafif gürültü ekleyerek çeşitliliği artır
])
print("✅ Veri artırma işlemi başarıyla tamamlandı!")


✅ Veri artırma işlemi başarıyla tamamlandı!


In [ ]:
# **BLOK: Normalizasyon İşlemi**
normalization_layer = tf.keras.layers.Rescaling(1./255)
print("✅ Normalizasyon işlemi başarıyla tamamlandı!")


✅ Normalizasyon işlemi başarıyla tamamlandı!


In [ ]:
# Normalizasyon işlemi
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Veri setine augmentasyon ve normalizasyon ekleme
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x), y))  # 1. Augmentation uygula
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))  # 2. Normalizasyon uygula

# Validation set sadece normalize edilmeli, augmentasyon uygulanmamalı!
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))

print("✅ Veri artırma ve normalizasyon işlemleri başarıyla tamamlandı!")

✅ Veri artırma ve normalizasyon işlemleri başarıyla tamamlandı!


In [ ]:
import tensorflow as tf
from tensorflow.keras.regularizers import l2  # L2 Regularization ekle

# RGB için giriş
rgb_input = tf.keras.Input(shape=(224, 224, 3), name="rgb_input")

# Önceden eğitilmiş ResNet50 modelini yükle
base_model_rgb = tf.keras.applications.ResNet50(
    include_top=False,  # Üst katmanları dahil etme
    input_shape=(224, 224, 3),
    weights="imagenet",  # ImageNet ön eğitimli ağırlıkları kullan
    name="rgb_resnet50"
)

# İlk 50 katmanı dondur
for layer in base_model_rgb.layers[:50]:  # İlk 50 katmanı dondur
    layer.trainable = False

# Özellik çıkarımı yap ve Global Average Pooling uygula
rgb_features = base_model_rgb(rgb_input)
rgb_features = tf.keras.layers.GlobalAveragePooling2D()(rgb_features)

# **İlk Dropout katmanı ekle (%50)**
rgb_features = tf.keras.layers.Dropout(0.5)(rgb_features)  # %50 Dropout

# **İkinci Dropout katmanı ekle (%50)**
rgb_features = tf.keras.layers.Dense(512, activation="relu")(rgb_features)  # Dense katman ekle
rgb_features = tf.keras.layers.Dropout(0.5)(rgb_features)  # %50 Dropout

# Çıkış katmanını oluştur
output = tf.keras.layers.Dense(num_classes, activation="softmax",
                               kernel_regularizer=l2(0.1))(rgb_features)  # L2 ekledi

# Modeli oluştur
rgb_model = tf.keras.Model(inputs=rgb_input, outputs=output)

# Model özetini yazdır
print("\n✅ RGB ResNet50 Modeli Başarıyla Oluşturuldu! Model Özeti:")
rgb_model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step

✅ RGB ResNet50 Modeli Başarıyla Oluşturuldu! Model Özeti:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rgb_input (InputLayer)               │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rgb_resnet50 (Functional)            │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 150)                 │          76,950 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,713,750 (94.28 MB)

 Trainable params: 24,053,142 (91.76 MB)

 Non-trainable params: 660,608 (2.52 MB)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# ReduceLROnPlateau Callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',  # Validation loss'u izle
    factor=0.5,  # Öğrenme oranını yarıya indir
    patience=3,  # 3 epoch boyunca iyileşme olmazsa learning rate’i azalt
    min_lr=1e-7,  # Öğrenme oranının aşırı düşmesini engelle
    cooldown=2,  # Learning rate düşürüldükten sonra 2 epoch boyunca sabit kalsın
    verbose=1  # Terminalde bilgileri göster
)

# EarlyStopping Callback (Patience süresi ile güncellendi)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,  # Erken durdurma süresi azaltıldı, 10 epoch boyunca iyileşme bekleniyor
    restore_best_weights=True
)

print("✅ ReduceLROnPlateau ve EarlyStopping callback’leri eklendi.")

✅ ReduceLROnPlateau ve EarlyStopping callback’leri eklendi.


In [ ]:
# Model derleme
print("\nModel derleniyor...")
rgb_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),  # Learning rate optimize edildi
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
print("✅ Model başarıyla derlendi ve kullanıma hazır!")



Model derleniyor...
✅ Model başarıyla derlendi ve kullanıma hazır!


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=30,  # Daha uzun bekleme süresi eklenerek erken durdurma geciktirildi
    restore_best_weights=True
)

history = rgb_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=40,  # Eğitim süresi 40 epoch olarak belirlenmiş
    batch_size=32,  # Batch size 32 olarak güncellendi
    callbacks=[early_stopping]  # EarlyStopping eklenmiş hali
)

print("\n✅ Model eğitimi tamamlandı! 🎉")


Epoch 1/40
560/560 ━━━━━━━━━━━━━━━━━━━━ 2426s 4s/step - accuracy: 0.0063 - loss: 26.0310 - val_accuracy: 0.0132 - val_loss: 19.5036
Epoch 2/40
560/560 ━━━━━━━━━━━━━━━━━━━━ 233s 415ms/step - accuracy: 0.0112 - loss: 17.8160 - val_accuracy: 0.0372 - val_loss: 13.3808
Epoch 3/40
560/560 ━━━━━━━━━━━━━━━━━━━━ 236s 421ms/step - accuracy: 0.0313 - loss: 12.3326 - val_accuracy: 0.0552 - val_loss: 9.5261
Epoch 4/40
560/560 ━━━━━━━━━━━━━━━━━━━━ 222s 396ms/step - accuracy: 0.0494 - loss: 8.9100 - val_accuracy: 0.1415 - val_loss: 6.7531
Epoch 5/40
560/560 ━━━━━━━━━━━━━━━━━━━━ 240s 428ms/step - accuracy: 0.0989 - loss: 6.7646 - val_accuracy: 0.1893 - val_loss: 5.4272
Epoch 6/40
560/560 ━━━━━━━━━━━━━━━━━━━━ 229s 408ms/step - accuracy: 0.1849 - loss: 5.3998 - val_accuracy: 0.3651 - val_loss: 4.0156
Epoch 7/40
560/560 ━━━━━━━━━━━━━━━━━━━━ 246s 440ms/step - accuracy: 0.2883 - loss: 4.4192 - val_accuracy: 0.5808 - val_loss: 2.9858
Epoch 8/40
560/560 ━━━━━━━━━━━━━━━━━━━━ 245s 437ms/step - accuracy: 0.400

In [ ]:
# Modeli kaydetme
model_save_path = '/content/drive/MyDrive/veriseti/rgb_model_resnet50.h5'
rgb_model.save(model_save_path) # Changed 'model' to 'rgb_model'
print(f"Model başarıyla {model_save_path} konumuna kaydedildi! ✅")

Model başarıyla /content/drive/MyDrive/veriseti/rgb_model_resnet50.h5 konumuna kaydedildi! ✅


TEST ASAMASI

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os


In [ ]:
# TensorFlow'dan model yükleme
from tensorflow.keras.models import load_model

# Modelin kaydedildiği dosya yolunu belirtin
# Check if this is the correct path and if the file exists.
# If not, update the path to the correct location.
model_path = '/content/drive/MyDrive/veriseti/rgb_model_resnet50.h5'

# Modeli yükleyin
rgb_model = load_model(model_path)

# Modelin başarıyla yüklendiğini belirtin

print("✅ ResNet50 modeli başarıyla yüklendi.")





✅ ResNet50 modeli başarıyla yüklendi.


In [ ]:
# Modeli tekrar derle
rgb_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
print("\n ✅ Model başarıyla derlendi ve kullanıma hazır! 🚀")



 ✅ Model başarıyla derlendi ve kullanıma hazır! 🚀


In [ ]:
def prepare_test_sets(data):
    print("Galeri Verileri:")
    gallery = data[data['camera'] == 'A']
    print(gallery)

    print("Probe B Verileri:")
    probe_B = data[data['camera'] == 'B']
    print(probe_B)

    print("Probe C Verileri:")
    probe_C = data[data['camera'] == 'C']
    print(probe_C)
    return gallery, probe_B, probe_C

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def preprocess_image(image_path, target_size=(224, 224)):
    """
    ResNet50 modeli için test görüntülerini hazırlar.
    - 224x224 boyutuna ölçeklendirir.
    - 0-1 arasına normalize eder.
    - Batch dimension ekler.
    """
    img = load_img(image_path, target_size=target_size)  # 📌 Resmi yükle ve boyutlandır
    img_array = img_to_array(img) / 255.0  # 📌 0-1 arasına normalize et
    img_array = np.expand_dims(img_array, axis=0)  # 📌 Batch dimension ekle
    return img_array

def extract_features_resnet50(model, root_dir, camera_label, batch_size=32):
    """
    ResNet50 modeli kullanarak belirtilen dizindeki tüm görüntülerden özellik çıkarır.
    - root_dir içindeki tüm .jpg ve .png dosyalarını işler.
    - Her görüntü için feature vektörünü hesaplar ve listeye ekler.
    - ID ve kamera bilgisini kaydeder.
    """
    features, ids, cameras = [], [], []
    batch_images = []
    batch_paths = []

    # 📌 Klasör içindeki tüm dosyaları tara
    for root, _, files in os.walk(root_dir):
        for file_name in files:
            if file_name.endswith('.jpg') or file_name.endswith('.png'):
                file_path = os.path.join(root, file_name)
                batch_images.append(preprocess_image(file_path))
                batch_paths.append(file_path)

                # 📌 Batch dolduğunda işle
                if len(batch_images) == batch_size:
                    batch_images_np = np.vstack(batch_images)
                    batch_features = model.predict(batch_images_np, batch_size=batch_size)

                    # 📌 Sonuçları listeye ekle
                    for i, feature in enumerate(batch_features):
                        features.append(feature.reshape(-1))
                        ids.append(os.path.basename(os.path.dirname(batch_paths[i])))
                        cameras.append(camera_label)

                    # 📌 Belleği temizle
                    batch_images.clear()
                    batch_paths.clear()

    # 📌 Kalanları işle (batch_size'den küçük kalanları)
    if batch_images:
        batch_images_np = np.vstack(batch_images)
        batch_features = model.predict(batch_images_np, batch_size=len(batch_images))

        for i, feature in enumerate(batch_features):
            features.append(feature.reshape(-1))
            ids.append(os.path.basename(os.path.dirname(batch_paths[i])))
            cameras.append(camera_label)

    print(f"Extracted {len(features)} features from '{root_dir}' ✅")
    return np.array(features), ids, cameras

# 📌 Güncellenmiş Örnek Kullanımı (ResNet50 Modeliyle)
a_dir = "/content/drive/MyDrive/veriseti/prcc/rgb/test/A/"
b_dir = "/content/drive/MyDrive/veriseti/prcc/rgb/test/B/"
c_dir = "/content/drive/MyDrive/veriseti/prcc/rgb/test/C/"

# 📌 Toplu işleme ile modelden özellik çıkar
features_A, ids_A, cameras_A = extract_features_resnet50(rgb_model, a_dir, 'A')
features_B, ids_B, cameras_B = extract_features_resnet50(rgb_model, b_dir, 'B')
features_C, ids_C, cameras_C = extract_features_resnet50(rgb_model, c_dir, 'C')


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [ ]:
import pandas as pd
import numpy as np

# ✅ Özellikleri birleştir (ResNet50)
all_features = np.concatenate([features_A, features_B, features_C], axis=0)
all_ids = ids_A + ids_B + ids_C
all_cameras = cameras_A + cameras_B + cameras_C

# ✅ Test veri setini oluştur
test_data = pd.DataFrame({
    'features': list(all_features),  # 🔥 ResNet50'dan gelen özellik vektörleri
    'id': all_ids,                  # 🔥 Kişi ID bilgisi
    'camera': all_cameras            # 🔥 Kamera bilgisi (A, B, C)
})

# ✅ Sonucu göster
print("✅ Test veri seti hazır.")
print(test_data.head())  # 🔥 İlk 5 satırı yazdır


✅ Test veri seti hazır.
                                            features   id camera
0  [0.0040341183, 0.014706066, 0.0002462314, 0.00...  001      A
1  [0.005668282, 0.03162106, 0.00013140318, 0.000...  001      A
2  [0.032185305, 0.005367751, 0.0003988363, 0.000...  001      A
3  [0.0065827337, 0.017362105, 0.00023668693, 0.0...  001      A
4  [0.0016561229, 0.01583296, 0.00010622789, 7.59...  001      A


In [ ]:
# ✅ Galeri ve sorgu (probe) setlerini oluştur (ResNet50 için)
gallery_rgb, probe_B_rgb, probe_C_rgb = prepare_test_sets(test_data)

# ✅ Galeri ve sorgu setlerini kontrol et ve ekrana yazdır
print("✅ **Galeri Set Örnekleri (ResNet50 RGB):**")
print(gallery_rgb.head())  # 🔥 İlk 5 satırı yazdır

print("\n✅ **Probe B Seti Örnekleri (ResNet50 RGB):**")
print(probe_B_rgb.head())  # 🔥 İlk 5 satırı yazdır

print("\n✅ **Probe C Seti Örnekleri (ResNet50 RGB):**")
print(probe_C_rgb.head())  # 🔥 İlk 5 satırı yazdır


Galeri Verileri:
                                               features   id camera
0     [0.0040341183, 0.014706066, 0.0002462314, 0.00...  001      A
1     [0.005668282, 0.03162106, 0.00013140318, 0.000...  001      A
2     [0.032185305, 0.005367751, 0.0003988363, 0.000...  001      A
3     [0.0065827337, 0.017362105, 0.00023668693, 0.0...  001      A
4     [0.0016561229, 0.01583296, 0.00010622789, 7.59...  001      A
...                                                 ...  ...    ...
3379  [3.332716e-05, 2.9488563e-06, 1.5832808e-07, 4...  322      A
3380  [0.0013241253, 0.00025278376, 1.5698282e-05, 0...  322      A
3381  [1.762363e-05, 9.658402e-07, 4.3819007e-08, 2....  322      A
3382  [2.3867515e-05, 1.4401752e-06, 8.2132104e-08, ...  322      A
3383  [0.00052750634, 0.00018356054, 1.1339728e-05, ...  322      A

[3384 rows x 3 columns]
Probe B Verileri:
                                               features   id camera
3384  [0.0009878714, 0.002953889, 5.860557e-05, 0.00... 

In [ ]:
import numpy as np

def rank_k_matching_resnet50(gallery_rgb, probe_rgb, k=1):
    """
    Rank-k doğruluk hesaplar.
    """
    correct_matches = 0

    for i in range(len(probe_rgb)):
        probe_feature = probe_rgb.iloc[i]['features']  # 📌 Probe RGB feature vektörü
        probe_id = probe_rgb.iloc[i]['id']  # 📌 Probe ID

        # 📌 Galeri özellikleriyle mesafeleri hesapla (Kosinüs benzerliği)
        distances = []
        for j in range(len(gallery_rgb)):
            gallery_feature = gallery_rgb.iloc[j]['features']  # 📌 Gallery RGB feature vektörü
            similarity = np.dot(probe_feature, gallery_feature) / (np.linalg.norm(probe_feature) * np.linalg.norm(gallery_feature))
            distances.append((similarity, gallery_rgb.iloc[j]['id']))  # 📌 Benzerlik ve ID'yi kaydet

        # 📌 En yüksek benzerlik skoruna göre sıralama yap
        distances = sorted(distances, key=lambda x: x[0], reverse=True)
        top_k_ids = [item[1] for item in distances[:k]]  # 📌 İlk k tahmin edilen ID'yi al

        # 📌 Eğer probe ID, ilk k içerisinde varsa doğru eşleşme
        if probe_id in top_k_ids:
            correct_matches += 1

    return correct_matches / len(probe_rgb)  # 📌 Rank-k doğruluk oranı

In [ ]:
# ✅ Rank-1 ve Rank-5 doğruluk hesaplaması (Ayrı Blok)
print("\n🔹 Rank-1 ve Rank-5 Doğruluk Oranları (ResNet50 RGB) 🔹\n")

rank_1_accuracy_B_rgb = rank_k_matching_resnet50(gallery_rgb, probe_B_rgb, k=1)
rank_5_accuracy_B_rgb = rank_k_matching_resnet50(gallery_rgb, probe_B_rgb, k=5)

rank_1_accuracy_C_rgb = rank_k_matching_resnet50(gallery_rgb, probe_C_rgb, k=1)
rank_5_accuracy_C_rgb = rank_k_matching_resnet50(gallery_rgb, probe_C_rgb, k=5)

print(f"✅ Rank-1 Accuracy (Gallery vs Probe B): {rank_1_accuracy_B_rgb:.4f}")
print(f"✅ Rank-5 Accuracy (Gallery vs Probe B): {rank_5_accuracy_B_rgb:.4f}")

print(f"✅ Rank-1 Accuracy (Gallery vs Probe C): {rank_1_accuracy_C_rgb:.4f}")
print(f"✅ Rank-5 Accuracy (Gallery vs Probe C): {rank_5_accuracy_C_rgb:.4f}")


🔹 Rank-1 ve Rank-5 Doğruluk Oranları (ResNet50 RGB) 🔹

✅ Rank-1 Accuracy (Gallery vs Probe B): 0.5210
✅ Rank-5 Accuracy (Gallery vs Probe B): 0.6690
✅ Rank-1 Accuracy (Gallery vs Probe C): 0.1140
✅ Rank-5 Accuracy (Gallery vs Probe C): 0.1660


In [ ]:
# ✅ Rank-10 ve Rank-20 doğruluk hesaplaması (Ayrı Blok)
print("\n🔹 Rank-10 ve Rank-20 Doğruluk Oranları (ResNet50 RGB) 🔹\n")

rank_10_accuracy_B_rgb = rank_k_matching_resnet50(gallery_rgb, probe_B_rgb, k=10)
rank_20_accuracy_B_rgb = rank_k_matching_resnet50(gallery_rgb, probe_B_rgb, k=20)

rank_10_accuracy_C_rgb = rank_k_matching_resnet50(gallery_rgb, probe_C_rgb, k=10)
rank_20_accuracy_C_rgb = rank_k_matching_resnet50(gallery_rgb, probe_C_rgb, k=20)

print(f"✅ Rank-10 Accuracy (Gallery vs Probe B): {rank_10_accuracy_B_rgb:.4f}")
print(f"✅ Rank-20 Accuracy (Gallery vs Probe B): {rank_20_accuracy_B_rgb:.4f}")

print(f"✅ Rank-10 Accuracy (Gallery vs Probe C): {rank_10_accuracy_C_rgb:.4f}")
print(f"✅ Rank-20 Accuracy (Gallery vs Probe C): {rank_20_accuracy_C_rgb:.4f}")


🔹 Rank-10 ve Rank-20 Doğruluk Oranları (ResNet50 RGB) 🔹

✅ Rank-10 Accuracy (Gallery vs Probe B): 0.7273
✅ Rank-20 Accuracy (Gallery vs Probe B): 0.7888
✅ Rank-10 Accuracy (Gallery vs Probe C): 0.1962
✅ Rank-20 Accuracy (Gallery vs Probe C): 0.2543
